In [1]:
from keras.preprocessing.image import load_img, save_img, img_to_array
import numpy as np
import scipy
import argparse

from keras.applications import inception_v3
from keras import backend as K

Using TensorFlow backend.


In [2]:
# disable all training operations since we won't be doing any training to the model.
K.set_learning_phase(0)

# download the pretrained Inception V3 model without its top part.
model = inception_v3.InceptionV3(weights='imagenet', include_top=False)

# print the model summary
model.summary()

87916544/87910968 [==============================] - 20s 0us/step
Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
_____________________

From there we need to define a dictionary that specifies which layers we’ll be using generate
the dream:

In [3]:
layer_contributions = {
                        'mixed2': 0.2,
                        'mixed3': 3.,
                        'mixed4': 2.,
                        'mixed5': 1.5,
                        }

In [4]:
# a dictionary that maps layer names to layer instances
layer_dict = dict([(layer.name, layer) for layer in model.layers])

# define the loss by adding layer contributions to this scalar variable.
loss = K.variable(0.)

for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    
    # Adds the L2 norm of the features of a layer to the loss. 
    # You avoid border artifacts by only involving nonborder pixels in the loss.
    loss = loss + coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling

In [5]:
# a tensor that holds the generated image
dream = model.input

# compute the gradients of the dream image with regard to the loss
grads = K.gradients(loss, dream)[0]

# normalize the gradients
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

# set up a keras function to retrieve the value of the loss and gradients given an input image
outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

# run the gradient ascent process for a number of iterations
def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...Loss value at', i, ':', loss_value)
        x += step * grad_values
    return x

In [6]:
from keras.preprocessing import image

def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img

def resize_img(img, size):
    img = np.copy(img)
    factors = (1,
               float(size[0]) / img.shape[1],
               float(size[1]) / img.shape[2],
            1)
    return scipy.ndimage.zoom(img, factors, order=1)
    
def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [7]:
import numpy as np
from keras.preprocessing.image import save_img

# Playing with these hyperparameters will also allow you to achieve new effects
step = 0.01
num_octave = 3
octave_scale = 1.4
iterations = 20
max_loss = 10.
base_image_path = 'GoldenGateBridge.jpg'

img = preprocess_image(base_image_path)

original_shape = img.shape[1:3]

successive_shapes = [original_shape]

for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    
    successive_shapes.append(shape)
    
successive_shapes = successive_shapes[::-1]

original_img = np.copy(img)

shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    
    print('Processing image shape', shape)
    
    img = resize_img(img, shape)
    img = gradient_ascent(img, iterations=iterations, step=step, max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img
    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)
    
    phil_img = deprocess_image(np.copy(img))
    save_img('deepdream_output/dream_at_scale_' + str(shape) + '.png', phil_img)

# save the result to disk
final_img = deprocess_image(np.copy(img))
save_img('final_dream.png', final_img)



Processing image shape (271, 407)
...Loss value at 0 : 1.5796549
...Loss value at 1 : 1.8665488
...Loss value at 2 : 2.3948474
...Loss value at 3 : 2.932391
...Loss value at 4 : 3.5347834
...Loss value at 5 : 4.176659
...Loss value at 6 : 4.7564487
...Loss value at 7 : 5.3389325
...Loss value at 8 : 5.884512
...Loss value at 9 : 6.4599853
...Loss value at 10 : 7.025447
...Loss value at 11 : 7.5905185
...Loss value at 12 : 8.088908
...Loss value at 13 : 8.6474
...Loss value at 14 : 9.146274
...Loss value at 15 : 9.617089
Processing image shape (380, 570)
...Loss value at 0 : 2.9290404
...Loss value at 1 : 4.2666817
...Loss value at 2 : 5.3194
...Loss value at 3 : 6.2591
...Loss value at 4 : 7.139002
...Loss value at 5 : 7.928192
...Loss value at 6 : 8.682022
...Loss value at 7 : 9.397178
Processing image shape (533, 799)
...Loss value at 0 : 3.000479
...Loss value at 1 : 4.162324
...Loss value at 2 : 5.229808
...Loss value at 3 : 6.1891165
...Loss value at 4 : 7.1040564
...Loss value at

### The End!